In [12]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import cv2
from imutils import paths
import random
import os
from keras.utils import to_categorical
#如果没有这两行代码，可能会报错OSError: image file is truncated
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [21]:
norm_size = 32
CLASS_NUM = 3
EPOCHS = 35
INIT_LR = 1e-2
BS = 32

In [4]:
#编写加载图像的函数
def load_data(path):
    print("[INFO] loading images...")
    data = []
    labels = []
    # grab the image paths and randomly shuffle them
    imagePaths = sorted(list(paths.list_images(path)))
    random.seed(42)
    random.shuffle(imagePaths)
    # loop over the input images
    for imagePath in imagePaths:
        img = image.load_img(imagePath, target_size=(224, 224))
        x = image.img_to_array(img)
        data.append(x)
        #label是上级目录名
        label = int(imagePath.split(os.path.sep)[-2])       
        labels.append(label)
    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)

    # convert the labels from integers to vectors
    labels = to_categorical(labels, num_classes=CLASS_NUM)                         
    return data,labels

In [20]:
#搭建网络
#加载预训练模型，需要下载预训练模型，可能会报错ConnectionResetError: [WinError 10054] 远程主机强迫关闭了一个现有的连接（网络原因）。
from keras.optimizers import Adam
from keras.models import Model
model = ResNet50()
#加一个dense层吧
# x_in = Input(shape=(None,))
# x = model([x_in])
# x = Lambda(lambda x: x[:, 0])(x)
# p = Dense(3, activation='softmax')(x)
# model = Model([x_in], p)

opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])

In [18]:
train_file_path = r'/openbayes/input/input0/train'
test_file_path = r'/openbayes/input/input0/trial'
trainX,trainY = load_data(train_file_path)
testX,testY = load_data(test_file_path)

[INFO] loading images...


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:802: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


[INFO] loading images...


In [22]:
from keras.preprocessing.image import ImageDataGenerator
#通过实时数据增强生成张量图像数据批次。数据将不断循环（按批次）。
#rotation_range: 整数。随机旋转的度数范围。
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
        height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True, fill_mode="nearest")

model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
        validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
        epochs=EPOCHS, verbose=1)

Epoch 1/35
200/200 [==============================] - 97s 486ms/step - loss: 1.1237 - accuracy: 0.5488 - val_loss: 0.8685 - val_accuracy: 0.6157
Epoch 2/35
200/200 [==============================] - 83s 417ms/step - loss: 0.9060 - accuracy: 0.5883 - val_loss: 0.8689 - val_accuracy: 0.6157
Epoch 3/35
200/200 [==============================] - 83s 416ms/step - loss: 0.9005 - accuracy: 0.5891 - val_loss: 0.8684 - val_accuracy: 0.6157
Epoch 4/35
200/200 [==============================] - 83s 416ms/step - loss: 0.8977 - accuracy: 0.5933 - val_loss: 0.8736 - val_accuracy: 0.6157
Epoch 5/35
200/200 [==============================] - 84s 418ms/step - loss: 0.9000 - accuracy: 0.5902 - val_loss: 1.4573 - val_accuracy: 0.6157
Epoch 6/35
200/200 [==============================] - 83s 416ms/step - loss: 0.9014 - accuracy: 0.5927 - val_loss: 0.8783 - val_accuracy: 0.6157
Epoch 7/35
200/200 [==============================] - 83s 413ms/step - loss: 0.8962 - accuracy: 0.5914 - val_loss: 0.8668 - val_ac

KeyboardInterrupt: 